In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load the data
df = pd.read_csv("laptop_data.csv")

In [3]:
#Lets filter the unwanted data,like monitors, tablet extra
df = df[df['CPU Speed'].str.contains('GHz')]

In [4]:
# Lets make the review feature as numeric
df['No. Reviews'] = df['No. Reviews'].str.replace(',','').astype(int)

In [5]:
#Removed irrelavant data and converted it to numeric
df['Display size'] = df[df['Display size'].str.contains('in')]['Display size'].str.replace('in','')
df = df.dropna()
df.drop(df[df['Display size'].str.contains('W')].index , inplace =True)
df['Display size'] = df['Display size'].astype(float)

In [6]:
# As there are mulitple values for CPU speed, its hard to create dummy columns. So I changed it to numeric
df['CPU Speed'] = df['CPU Speed'].str.replace('GHz','').astype(float)

In [7]:
#Dropped duplicate rows
df.drop(df[df['RAM'].str.contains('-')][1:].index, inplace = True)

In [8]:
#Cleaning the RAM feature
df.RAM = df.RAM.str.replace('GB','')
df.RAM = df.RAM.str.replace('TB','')

df.RAM = df.RAM.replace( '-' , np.NAN)
df.RAM = df.RAM.fillna(df['RAM'].mode()[0]).astype(int)

In [9]:
#Cleaning the ratings column
df.Rating = df['Rating'].str[:3].astype(float)

In [10]:
#Fixing the storage types
df['Storage type'] = df['Storage type'].str.replace('Solid State Drive','SSD')
df['Storage type'] = df['Storage type'].apply(lambda x : 'SSD' if ('ssd' in x.lower()) else x )

#Fixed the EMMC
df['Storage type'] = df['Storage type'].apply(lambda x : 'EMMC' if ('emmc' in x.lower()) else x )
df['Storage type'] = df['Storage type'].apply(lambda x : 'EMMC' if ('Embedded MultiMediaCard' in x) else x )

#Fixed the HDD
df['Storage type'] = df['Storage type'].apply(lambda x : 'HDD' if ('hdd' in x.lower()) else x )

#Replacing the rest of the data with others
df['Storage type'] = df['Storage type'].replace('Mechanical Hard Drive','Others').replace('Hybrid Drive','Others')
df['Storage type'] = df['Storage type'].replace('-','Others').replace('Integrated','Others').replace('Hybrid','Others').replace('SDD','Others')
df['Storage type'] = df['Storage type'].replace('64gb','Others').replace('Flash','Others').replace('sshd','Others').replace('USB','Others')

In [11]:
#Extracting brand from the title
df['Brand'] = df['Title'].apply(lambda x : x.split(' ')[:][0])

In [12]:
#Dropped the duplicate values
df.drop(df[df['Brand'].str.contains("14.1")][1:].index, inplace = True)

In [13]:
#Again dropped the duplicate value, I think there were many same sponsered ads
df.drop_duplicates(subset='Title',keep="first", inplace = True)

In [14]:
df.Brand = df.Brand.str.replace('BLACK','Hp')

In [30]:
pd.options.display.max_colwidth = 250
df.RAM[998] = 8
df.Brand = df.Brand.apply(lambda x : x.lower() )

<ipython-input-30-3bead77bada0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.RAM[998] = 8


In [44]:
# Now lets only keep the important brands and replace the rest with other_brand
def brand_maker(x):
    if (x=="lenovo") or (x=="hp") or (x=="asus") or (x=="dell") or (x=="apple") or (x=="acer") :
        return x
    else:
        return 'other_brand'

In [53]:
df.Brand = df.Brand.apply(brand_maker)

In [108]:
#Lets make the title to lower, so we can extract more informartion with ease
#Lets check if it has any gaming laptops, which typically comes with a powerful graphics cards, which might be useful to extract
index_for_grp = df[df.Title.str.lower().str.contains('graphics|gaming|nvidia')].index

In [109]:
#If it has graphics card set it to 1, 0 is default
df['graphics'] = 0
df.loc[index_for_grp,'graphics'] = 1

In [118]:
i5_index = df[df.Title.str.lower().str.contains('i5')].index
i7_index = df[df.Title.str.lower().str.contains('i7')].index
i9_index = df[df.Title.str.lower().str.contains('i9')].index
i3_index = df[df.Title.str.lower().str.contains('i3')].index
amd_index = df[df.Title.str.lower().str.contains('amd')].index

In [119]:
df["Processor"] = 'unknown'

In [122]:
df.loc[i5_index,'Processor'] = 'i5'
df.loc[i7_index,'Processor'] = 'i7'
df.loc[i9_index,'Processor'] = 'i9'
df.loc[i3_index,'Processor'] = 'i3'
df.loc[amd_index,'Processor'] = 'amd'

#### There are 9 M1 chips and i left them as unknown, cause creating a new column for 9 values, does'nt seem important
#### Just looked there are some laptops with wrong RAM values, just have to manually update it. Its painful :\

In [141]:
df[df.RAM == 1]

,Title,Price,No. Reviews,Display size,Storage type,CPU Speed,RAM,Rating,Brand,graphics,Processor
197,"FANCY CHERRY 10 inch 8GB Laptop Netbook Notebook PC Ultrabook Android HDMI Dual Core WIFI Camera, Accessories Laptop Bag + Mouse + Mouse Pad + Earphone (Black)",119.00,246,10.0,SSD,1.5,1,3.0,other_brand,0,unknown
296,"Dell G7 17.3 inch FHD 144Hz 300 nits IPS Anti-Glare LED Backlit Narrow Border Display Gaming laptop, Intel Core i7-10750H, 16 GB RAM, 1 TB SSD, NVIDIA GeForce RTX 2060 6GB GDDR6, Win 10 Home",1469.00,13,17.3,Flash Memory Solid State,5.0,1,4.1,dell,1,i7
315,FANCY CHERRY® NEW 2018 HD 10 inch Mini Laptop Notebook Netbook Tablet Computer 1G DDR3 8GB Memory VIA WM8880 CPU Dual Core Android Screen Wifi Camera Keyboard USB HDMI (Silver 8GB),109.00,241,10.0,SSD,1.5,1,3.0,other_brand,0,unknown
344,"FANCY CHERRY 10 inch 8GB Laptop Netbook Notebook PC Ultrabook Android HDMI Dual Core WIFI Camera, Accessories Laptop Bag + Mouse + Mouse Pad + Earphone (White)",118.14,246,10.0,SSD,1.5,1,3.0,other_brand,0,unknown
474,"G-Anica® Netbook Laptop PC 10 inch Android Portable Ultrabook,Dual Core, Wifi,with Laptop Bag + Mouse + Mouse Pad + Earphone (4 PCS Computer Accessories) (Pink)",139.99,153,10.0,HDD,1.5,1,3.0,other_brand,0,unknown
506,"FANCY CHERRY 10 inch 8GB Laptop Netbook Notebook PC Ultrabook Android HDMI Dual Core WIFI Camera, Accessories Laptop Bag + Mouse + Mouse Pad + Earphone (Pink)",119.00,246,10.0,SATA,1.5,1,3.0,other_brand,0,unknown
553,"FANCY CHERRY 10 inch 8GB Laptop Netbook Notebook PC Ultrabook Android HDMI Dual Core WIFI Camera, Accessories Laptop Bag + Mouse + Mouse Pad + Earphone (Silver)",118.14,246,10.0,SATA,1.5,1,3.0,other_brand,0,unknown
669,"G-Anica® Netbook Laptop PC 10 inch Android Portable Ultrabook,Dual Core, Wifi,with Laptop Bag + Mouse + Mouse Pad + Earphone (4 PCS Computer Accessories) (Black)",139.99,153,10.0,HDD,1.5,1,3.0,other_brand,0,unknown
710,FANCY CHERRY® NEW 2018 HD 10 inch Mini Laptop Notebook Netbook Tablet Computer 1G DDR3 8GB Memory VIA WM8880 CPU Dual Core Android Screen Wifi Camera Keyboard USB HDMI (Black 8GB),109.00,241,10.0,SSD,1.5,1,3.0,other_brand,0,unknown
774,BIGMAC 10.1 Inch Quad Core 8GB Computer Laptop PC Android 6.0 Mini Netbook Slim and Lightweight Notebook WiFi Webcam Netflix YouTube Google Player Flash (Pink),143.50,114,10.1,SSD,1.5,1,3.0,other_brand,0,unknown
